# Pinpoint suspicious financial transactions with Oracle Spatial and Python

In this workshop you will identify suspicious financial transactions based on analysis involving location and time ("spatiotemporal"). Both the spatial features of Oracle Database (i.e., "Oracle Spatial") and Python libraries are used. Oracle Spatial performs back-end large scale location analysis and enrichment based on relationships and measurements such as proximity, containment, distance and area. Python provides an environment for data analysis and machine learning leveraging a vast ecosystem of mature specialized libraries, including spatiotemporal analysis. The python-oracledb driver enables connectivity and robust access to Oracle Database, so that the strengths of both the Oracle back-end and Python client are optimally leveraged.

## Connect to Autonomous Database from Python
To prepare for data loading and analysis, you first establish a connection from Python to your Autonomous Database. The python-oracledb driver supports this connection and all subsequent database interactions. You will use the python-oracledb driver's ‘Thin’ mode which connects directly to Oracle Database and does not need Oracle Client libraries.

### Load the python-oracedb module
In the first cell, run the following statement. This loads the python-oracedb module which handles interaction with Oracle Database.

In [1]:
import oracledb

### Load your ADB password and DSN into variables
Run the following statements. This loads your ADB password and DSN into variables

In [2]:
import os

os.environ["TNS_ADMIN"] = "/Users/pparkins/Downloads/Wallet_financialdb"

In [ ]:
# Get ADB password and DSN from file
my_pwd = open('./my-pwd.txt','r').readline().strip()
my_dsn = open('./my-dsn.txt','r').readline().strip()

### Create a connection to your ADB
Run the following statements. This creates a connection to your ADB.

In [1]:
# Create database connection and cursor
import os
print(os.getcwd())
print(os.listdir("/Users/pparkins/Downloads/Wallet_financialdb"))
os.environ["TNS_ADMIN"] = "/Users/pparkins/Downloads/Wallet_financialdb"

import oracledb

# Set the wallet directory
os.environ["TNS_ADMIN"] = "/Users/pparkins/Downloads/Wallet_financialdb"
if not os.path.exists(os.path.join(os.environ["TNS_ADMIN"], "tnsnames.ora")):
    raise FileNotFoundError("tnsnames.ora not found in TNS_ADMIN directory")
# Now connect
connection = oracledb.connect(
    user="financial",
    password="Welcome12345",
    dsn="financialdb_high"
)

cursor = connection.cursor()
print("Connected successfully!")

/Users/pparkins/src/github.com/paulparkinson/oracle-ai-for-sustainable-dev/financial/suspicious-purchases-oml-spatial
['sqlnet.ora', 'createSecretFromWalletMac.sh', 'ewallet.pem', 'tnsnames.ora', 'README', 'keystore.jks', 'cwallet.sso', 'ewallet.p12', 'truststore.jks', 'ojdbc.properties']
Connected successfully!


### Test connection to your ADB
Run the following statements. This runs a test query to verify successful connection to ADB.

In [2]:
# Run a test query
cursor.execute("select object_type, count(*) from all_objects group by object_type")
for row in cursor.fetchmany(size=10):
  print(row)

('EDITION', 1)
('TABLE', 141)
('SYNONYM', 27854)
('INDEX', 92)
('SEQUENCE', 19)
('PACKAGE', 751)
('VIEW', 2603)
('FUNCTION', 352)
('PROCEDURE', 89)
('TYPE', 19813)


## Prepare Data
Load fictitious financial transactions data to your Autonomous Database and configure for spatial and temporal ("spatiotemporal") analysis.
### Create and Load Tables
Run the following cells to create `locations` and `transactions` tables, and load them with data from the `locations.csv` and `transactions.csv` files. Then, query the tables to ensure that data was loaded successfully, and preview the dataset.

In [3]:
# Create table for locations data
cursor.execute("""
 CREATE TABLE locations (
           location_id INTEGER, 
           owner VARCHAR2(100),  
           lon NUMBER, 
           lat NUMBER)""")

In [4]:
# Load the locations data
import csv
BATCH_SIZE = 1000
with connection.cursor() as cursor:
    with open('locations.csv', 'r') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        #skip header
        next(csv_reader) 
        #load data
        sql = "INSERT INTO locations VALUES (:1, :2, :3, :4)"
        data = []
        for line in csv_reader:
            data.append((line[0], line[1], line[2], line[3]))
            if len(data) % BATCH_SIZE == 0:
                cursor.executemany(sql, data)
                data = []
        if data:
            cursor.executemany(sql, data)
        connection.commit()

In [ ]:
# Preview locations data
cursor = connection.cursor()
cursor.execute("SELECT * FROM locations")
for row in cursor.fetchmany(size=10):
    print(row)

In [7]:
# Create table for transactions data
cursor.execute("""
   CREATE TABLE transactions (
                  trans_id INTEGER,
                  location_id INTEGER, 
                  trans_date DATE, 
                  cust_id INTEGER)""")

In [10]:
# Load the transactions data
BATCH_SIZE = 1000
with connection.cursor() as cursor:
    with open('transactions.csv', 'r') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        #skip header
        next(csv_reader) 
        #load data
        sql = "INSERT INTO transactions VALUES (:1, :2, TO_DATE(:3,'YYYY-MM-DD:HH24:MI:SS'), :4)"
        data = []
        for line in csv_reader:
            data.append((line[0], line[1], line[2], line[3]))
            if len(data) % BATCH_SIZE == 0:
                cursor.executemany(sql, data)
                data = []
        if data:
            cursor.executemany(sql, data)
        connection.commit()

In [11]:
# Preview transactions data
cursor = connection.cursor()
cursor.execute("SELECT * FROM transactions")
for row in cursor.fetchmany(size=10):
    print(row)

(529, 1250747966, datetime.datetime(2023, 2, 1, 15, 12, 57), 3)
(530, 1250748031, datetime.datetime(2023, 2, 4, 19, 25, 51), 3)
(531, 1254086055, datetime.datetime(2023, 2, 4, 15, 52, 26), 3)
(532, 1479752722, datetime.datetime(2023, 2, 1, 12, 9, 12), 3)
(533, 3155435317, datetime.datetime(2023, 2, 2, 3, 5, 24), 3)
(534, 8229497844, datetime.datetime(2023, 2, 6, 6, 16, 20), 3)
(535, 8347091110, datetime.datetime(2023, 2, 5, 19, 34, 5), 3)
(536, 8397911123, datetime.datetime(2023, 2, 3, 6, 11, 28), 3)
(537, 10675471918, datetime.datetime(2023, 2, 5, 16, 48, 1), 3)
(538, 1227587493, datetime.datetime(2023, 2, 3, 5, 31, 46), 3)


In [12]:
# Get distinct Customer ID's
cursor = connection.cursor()
cursor.execute("SELECT DISTINCT cust_id FROM transactions ORDER BY cust_id")
for row in cursor.fetchall():
    print(row[0])

1
2
3
4
5
6
7
8
9
10


### Add Epoch Date
Temporal calculations are a key component of this workshop, and are best performed on an integer representation of date and time. This integer representation is generally referred to as epoch time or more specifically UNIX time. In this task you add epoch time for all transactions.

#### Add and populate epoch date column
Run the following cells to add and populate a column for epoch date. Then, preview the transaction data, and observe the epoch date column is added.

In [13]:
# add column for epoch date
cursor.execute("ALTER TABLE transactions ADD (trans_epoch_date integer)")

In [14]:
# populate epoch date
cursor.execute("""UPDATE transactions 
                  SET trans_epoch_date = (trans_date - date'1970-01-01') * 86400""")
connection.commit()

In [15]:
# Preview transactions data
cursor.execute("SELECT * FROM transactions")
for row in cursor.fetchmany(size=10):
    print(row)

(529, 1250747966, datetime.datetime(2023, 2, 1, 15, 12, 57), 3, 1675264377)
(530, 1250748031, datetime.datetime(2023, 2, 4, 19, 25, 51), 3, 1675538751)
(531, 1254086055, datetime.datetime(2023, 2, 4, 15, 52, 26), 3, 1675525946)
(532, 1479752722, datetime.datetime(2023, 2, 1, 12, 9, 12), 3, 1675253352)
(533, 3155435317, datetime.datetime(2023, 2, 2, 3, 5, 24), 3, 1675307124)
(534, 8229497844, datetime.datetime(2023, 2, 6, 6, 16, 20), 3, 1675664180)
(535, 8347091110, datetime.datetime(2023, 2, 5, 19, 34, 5), 3, 1675625645)
(536, 8397911123, datetime.datetime(2023, 2, 3, 6, 11, 28), 3, 1675404688)
(537, 10675471918, datetime.datetime(2023, 2, 5, 16, 48, 1), 3, 1675615681)
(538, 1227587493, datetime.datetime(2023, 2, 3, 5, 31, 46), 3, 1675402306)


### Configure data for Spatial Operations
Spatial calculations are an additional key component of this workshop. In this task you configure your locations data to utilize the spatial features of Autonomous Database. The locations table includes longitude/latitude coordinates. One option is to create and populate a new column using the native spatial data type. While that would work perfectly fine, there is another option that takes advantage of a mainstream Oracle Database feature called "function-based indexing". This approach allows for all of the capability associated with creating a new spatial column, but without having to create the column. Instead, you create a database function that converts coordinates to a spatial data element, and then create an index on that function. Once the function and index are created, all spatial operations behave as if a new spatial column had been created. While this is not essential for the small data volume in this workshop, the approach is of great benefit for large scale systems where the overhead of adding a column is significant.

#### Create SQL function lonlat_to_proj_geom( ) 
Run the following cells to to create and test a function that converts longitude/latitude coordinates to Oracle's native spatial data type (i.e. SDO_GEOMETRY, referred to as a "geometry"). Not only does the function convert coordinates to the native spatial type, but it also converts the coordinates from longitude/latitude to a projected coordinate system with x/y coordinates in meters. This coordiate system coversion is required by Python libraries used in subsequent labs, hence it is convenient to perform this conversion in this function.

Run the following cells to create and test the function.

In [16]:
# Create function to return lon/lat coordinates as a geometry.
cursor.execute("""
 CREATE OR REPLACE FUNCTION lonlat_to_proj_geom (longitude IN NUMBER, latitude IN NUMBER)
 RETURN SDO_GEOMETRY DETERMINISTIC IS
 BEGIN
   IF latitude IS NULL OR longitude IS NULL
   OR latitude NOT BETWEEN -90 AND 90
   OR longitude NOT BETWEEN -180 AND 180
   THEN
     RETURN NULL;
   ELSE
      RETURN sdo_cs.transform(
        SDO_GEOMETRY(2001, 4326,
                     sdo_point_type(longitude, latitude, NULL),NULL, NULL),
        3857);
   END IF;
END;""")

In [10]:
# return LOBs directly as strings or bytes
oracledb.defaults.fetch_lobs = False

In [7]:
# test the function
cursor.execute("""
 with x as (
    SELECT location_id, lonlat_to_proj_geom(lon,lat) as geom FROM locations)
 SELECT location_id, geom, (geom).get_wkt()
 FROM x
 """)
for row in cursor.fetchone():
    print(row)

5413066421
<oracledb.DbObject MDSYS.SDO_GEOMETRY at 0x3045f36b0>
POINT (-10026261.0012563 3497754.03954987)


### Create Spatial Index
Spatial queries rely on a spatial index for optimal performance. A spatial index can only be created on data having uniform dimensionality (i.e., 2D or 3D) and coordinate system. Before creating a spatial index, it is necessary to insert a row of metadata describing these properties for the geometry to be indexed. This includes the table name, geometry column name (or in this case a function returning geometry), dimensionality , and a coordinate system code. When creating a spatial index, the data are first verified to conform to the metadata. Spatial indexing completes successfully only if the data conform to the metadata. 

Run the following cells to insert a row of metadata, create a spatial index, and run a spatial query to verify the spatial index.The spatial query returns the 5 nearest items from the locations table to a longitude/latitude coordinate, along with the distances. This is referred to as a "nearest neighbor" query and uses the sdo_nn( ) operator which uses the spatial index. For more info on nearest neighbor queries, please see the [documentation](https://docs.oracle.com/en/database/oracle/oracle-database/19/spatl/spatial-operators-reference.html#GUID-41E6B1FA-1A03-480B-996F-830E8566661D).

In [8]:
# insert spatial metadata
cursor.execute("""
 INSERT INTO user_sdo_geom_metadata VALUES (
    'LOCATIONS', 'ADMIN.LONLAT_TO_PROJ_GEOM(LON,LAT)',
     SDO_DIM_ARRAY(SDO_DIM_ELEMENT('LON', 0, 0, 0.05),
                   SDO_DIM_ELEMENT('LAT', 0, 0, 0.05)),
     3857)
            """)

DatabaseError: ORA-13223: duplicate entry for LOCATIONS.ADMIN.LONLAT_TO_PROJ_GEOM(LON,LAT) in SDO_GEOM_METADATA
ORA-06512: at "MDSYS.MD", line 1723
ORA-06512: at "MDSYS.MDERR", line 17
ORA-06512: at "MDSYS.SDO_GEOM_TRIG_INS1", line 91
ORA-04088: error during execution of trigger 'MDSYS.SDO_GEOM_TRIG_INS1'
Help: https://docs.oracle.com/error-help/db/ora-13223/

In [9]:
# create spatial index
cursor.execute("""
 CREATE INDEX locations_sidx
 ON locations(LONLAT_TO_PROJ_GEOM(LON,LAT))
 INDEXTYPE IS mdsys.spatial_index_v2
            """)

DatabaseError: ORA-00955: name is already used by an existing object
Help: https://docs.oracle.com/error-help/db/ora-00955/

In [3]:
# run test query that uses spatial index
cursor.execute("""
 SELECT location_id, round(sdo_nn_distance(1), 2) FROM locations
 WHERE sdo_nn(
   LONLAT_TO_PROJ_GEOM(LON,LAT),
   LONLAT_TO_PROJ_GEOM( -97.6, 30.3),
   'sdo_num_res=5 unit=mile', 1) = 'TRUE' """)
for row in cursor.fetchmany():
    print(row)

(280232499, 9.36)
(8369348344, 6.89)
(3855747359, 8.87)
(3852014144, 8.4)
(4933514996, 9.15)


## Explore Data
Explore the locations and transactions data prepared in the previous steps. By managing the data in Autonomous Database, you are able to perform back-end processing and analysis operations and then bring appropriate data subsets into Python for specialized analyses.

### Spatial data handling in Python
The most common Python library for data handling is Pandas, which provides DataFrame as the data structure akin to a table with columns and rows. The GeoPandas library extends Pandas for spatial data handling, where DataFrame is extended to GeoDataFrame including a "geometry" column. The Shapely library provides the spatial type used to populate the geometry column. Folium is a popular map visualization library and is used by GeoPandas.

In the following cells, import the libraries for spatial data handling and map visualization, and run throug ha simple example of spatial data in Python. These cells will create a GeoDataFrame containing point locations for several cities. The geometry values are in Well-Known Text ("WKT") format since that is the format used in a GeoDataFrame. Then, visualize the data by specifying both the background map and marker size. Move your mouse over a map marker to see its attributes.

In [4]:
import geopandas as gpd
import shapely
import folium

In [5]:
# create a GeoDataFrame
gdf = gpd.GeoDataFrame(
  {
    "city": ["Buenos Aires", "Brasilia", "Santiago", "Bogota", "Caracas"],
    "country": ["Argentina", "Brazil", "Chile", "Colombia", "Venezuela"],
    "geometry": ["POINT(-58.66 -34.58)",
                 "POINT(-47.91 -15.78)",
                 "POINT(-70.66 -33.45)",
                 "POINT(-74.08 4.60)",
                 "POINT(-66.86 10.48)",
        ],})
gdf["geometry"] = gpd.GeoSeries.from_wkt(gdf["geometry"])
gdf.set_geometry("geometry", inplace=True)
gdf.set_crs("EPSG:4326", inplace=True)
gdf

,city,country,geometry
0,Buenos Aires,Argentina,POINT (-58.66 -34.58)
1,Brasilia,Brazil,POINT (-47.91 -15.78)
2,Santiago,Chile,POINT (-70.66 -33.45)
3,Bogota,Colombia,POINT (-74.08 4.6)
4,Caracas,Venezuela,POINT (-66.86 10.48)


In [6]:
# render GeoDataFrame on an interactive map 
gdf.explore(tiles="CartoDB positron", marker_kwds={"radius":8})

#### Convert from the native spatial type to common formats
Oracle Spatial includes functions and methods to convert from the native spatial type to common formats, including conversion to the WKT format used in a GeoDataFrame. So creating a GeoDataFrame from Oracle Spatial results is straightforward. The conversion syntax of object methods is more compact than the equivalent SQL functions. For example the method `(geometry).get_wkt()` versus the function `sdo_util.to_wktgeometry(geometry)`. Run the following to see a basic example of format conversions of a hard-coded SDO_GEOMETRY to WKT and GeoJSON formats using object methods.

In [7]:
# convert native geometry data type to common string formats
cursor = connection.cursor()
cursor.execute("""
  WITH x AS (
    SELECT sdo_geometry(2001,4326,sdo_point_type(-100.12, 22.34,null),null,null) 
           as geometry
    FROM dual)
  SELECT geometry, 
         (geometry).get_wkt(), 
         (geometry).get_geojson()
  FROM x
  """)
for row in cursor.fetchone():
   print(row)

<oracledb.DbObject MDSYS.SDO_GEOMETRY at 0x12b4a2990>
POINT (-100.12 22.34)
{ "type": "Point", "coordinates": [-100.12, 22.34] }


#### Retrieve geometries using the function lonlat_to_proj_geom( )
In previous steps you configured the LOCATIONS table with a function-based spatial index. The function is lonlat_to_proj_geom( ) and converts longitude, latitude into a SDO_GEOMETRY in the World Mercator coordinate system for compatibility with libraries used in a later lab. Run the following cell to retrieve geometries using that function as WKT format and preivew the data. Then, retrieve geometries using that function and create a GeoDataFrame. Finally, visualize the GeoDataFrame and mouse over locations to see their attributes.

In [8]:
# convert geometries in locations table to WKT format
cursor = connection.cursor()
cursor.execute("""
  SELECT lon, lat, (lonlat_to_proj_geom(lon,lat)).get_wkt()
  FROM locations
  """)
for row in cursor.fetchmany(10):
   print(row)

(-90.067435, 29.9549005, <oracledb.LOB object at 0x30c80b2f0>)
(-104.772892, 39.7833465, <oracledb.LOB object at 0x30c80b440>)
(-77.04212, 38.9232218, <oracledb.LOB object at 0x30c80b4d0>)
(-104.941159, 39.6791465, <oracledb.LOB object at 0x30c792690>)
(-90.0521381, 29.9692125, <oracledb.LOB object at 0x30c800650>)
(-77.3068219, 38.6541807, <oracledb.LOB object at 0x30c813110>)
(-105.250259, 39.9842863, <oracledb.LOB object at 0x30c812240>)
(-97.7411049, 30.3617325, <oracledb.LOB object at 0x30c8121e0>)
(-104.7655575, 39.5197581, <oracledb.LOB object at 0x30c8120f0>)
(-97.734911, 30.2915028, <oracledb.LOB object at 0x30c812c60>)


In [14]:
# create a GeoDataFrame from the locations table
cursor.execute("""
 SELECT location_id, owner, (lonlat_to_proj_geom(lon,lat)).get_wkt()
 FROM locations
 """)
gdf = gpd.GeoDataFrame(cursor.fetchall(), columns = ['location_id', 'owner', 'geometry'])
gdf['geometry'] = gdf['geometry'].apply(str)
gdf['geometry'] = shapely.from_wkt(gdf['geometry'])
gdf = gdf.set_geometry('geometry')
gdf.set_crs("EPSG:3857", inplace=True)
gdf.head()


/var/folders/j2/4sbkqm5j54333s2y3r_2fsfw0000gq/T/ipykernel_25578/3027121760.py:7: UserWarning: Geometry column does not contain geometry.
  gdf['geometry'] = gdf['geometry'].apply(str)


,location_id,owner,geometry
0,5413066421,Chase,POINT (-10026261.001 3497754.04)
1,2293201913,Chase,POINT (-11663264.986 4834508.55)
2,4070361591,PNC Bank,POINT (-8576289.568 4710679.714)
3,960470070,Wells Fargo,POINT (-11681996.383 4819425.684)
4,10218511215,None,POINT (-10024558.158 3499593.011)


In [13]:
# render the GeoDataFrame on a map
gdf.explore(tiles="CartoDB positron")

### Explore Transactions Data

Run the following to create a GeoDataFrame from a query joining TRANSACTIONS to LOCATIONS. Then, visualize the GeoDataFrame. Mouse over an item to see transaction attributes.

In [17]:
# Create a geoDataFrame from a join of transactions and locations
cursor = connection.cursor()
cursor.execute("""
 SELECT a.cust_id, a.trans_id, a.trans_epoch_date, 
  (lonlat_to_proj_geom(b.lon,b.lat)).get_wkt() 
 FROM transactions a, locations b
 WHERE a.location_id=b.location_id
 """)
gdf = gpd.GeoDataFrame(cursor.fetchall(), columns = ['cust_id', 'trans_id', 'trans_epoch_date', 'geometry'])
gdf['geometry'] = shapely.from_wkt(gdf['geometry'])
gdf = gdf.set_geometry('geometry')  # or gdf.set_geometry('geometry', inplace=True)
gdf.set_crs("EPSG:3857", inplace=True)
gdf.head()

,cust_id,trans_id,trans_epoch_date,geometry
0,3,529,1675264377,POINT (-10877760.874 3555807.192)
1,3,530,1675538751,POINT (-10876312.774 3555139.341)
2,3,531,1675525946,POINT (-10878247.641 3554083.081)
3,3,532,1675253352,POINT (-10876729.966 3556996.203)
4,3,533,1675307124,POINT (-10878453.849 3555786.501)


In [18]:
# render the GeoDataFrame on a map
gdf.explore(tiles="CartoDB positron")

### Experiment with Spatial Aggregation
To calculate the distance of transactions from a spatiotemporal cluster, it is convenient to represent the cluster as a single geometry. This is a use case for spatial aggregation, where a set of geometries is represented by a single aggregate. Oracle Spatial provides a package of spatial aggregate functions for just this purpose. This task is meant to familiarize you with spatial aggregation.

#### Create a GeoDataFrame of items from the LOCATIONS table
Run the following cell to create a GeoDataFrame of items from the LOCATIONS table locations within 10 miles of a longitude/latitude coordinate in Austin, TX (-97.7431, 30.2672).

In [19]:
# create a GeoDataFrame of locations near a coordinate
cursor = connection.cursor()
cursor.execute("""
 SELECT (lonlat_to_proj_geom(lon,lat)).get_wkt() as geometry
 FROM locations
 WHERE sdo_within_distance(
           lonlat_to_proj_geom(lon,lat),
           lonlat_to_proj_geom(-97.7431,30.2672),
           'distance=10 unit=MILE') = 'TRUE'
       """)
gdfPoints = gpd.GeoDataFrame(cursor.fetchall(), columns = ['geometry'])
gdfPoints['geometry'] = shapely.from_wkt(gdfPoints['geometry'])
gdf = gdf.set_geometry('geometry')
gdf.set_crs("EPSG:3857", inplace=True)
gdfPoints.head()

,geometry
0,POINT (-10872367.066 3529555.461)
1,POINT (-10878109.838 3553778.272)
2,POINT (-10875873.007 3542471.146)
3,POINT (-10875678.019 3550170.141)
4,POINT (-10883186.975 3538915.882)


#### Create a GeoDataFrame in the center of the previously selected locations
Create a GeoDataFrame containing the location in the center of the previously selected locations. This location is referred to as an "aggregate centroid", hence the GeoDataFrame is named gdfAggCent.

In [21]:
# create a GeoDataFrame of the aggregate centroid
cursor.execute("""
 SELECT SDO_AGGR_CENTROID(
          SDOAGGRTYPE(lonlat_to_proj_geom(lon,lat), 0.005)).get_wkt() as geometry
 FROM locations
 WHERE sdo_within_distance(
           lonlat_to_proj_geom(lon,lat),
           lonlat_to_proj_geom(-97.7431,30.2672),
           'distance=10 unit=MILE') = 'TRUE'
       """)
gdfAggCent = gpd.GeoDataFrame(cursor.fetchall(), columns = ['geometry'])
gdfAggCent['geometry'] = shapely.from_wkt(gdfAggCent['geometry'])
gdfAggCent = gdfAggCent.set_geometry('geometry')
gdfAggCent.set_crs("EPSG:3857", inplace=True)
gdfAggCent

,geometry
0,POINT (-10882767.904 3538221.353)


#### Create a GeoDataFrame containing the shape that bounds the locations near the coordinate in Austin, TX
Create a GeoDataFrame containing the shape that bounds the locations near the coordinate in Austin, TX. This is referred to as a "aggregate convex hull", hence the GeoDataFrame is named gdfAggHull. There are several other spatial aggregate functions that follow the same pattern.

In [23]:
# create a GeoDataFrame of the bounding area
cursor.execute("""
 SELECT SDO_AGGR_CONVEXHULL(
          SDOAGGRTYPE(lonlat_to_proj_geom(lon,lat), 0.005)).get_wkt() as geometry
 FROM locations
 WHERE sdo_within_distance(
           lonlat_to_proj_geom(lon,lat),
           lonlat_to_proj_geom(-97.7431,30.2672),
           'distance=10 unit=MILE') = 'TRUE'
       """)
gdfAggHull = gpd.GeoDataFrame(cursor.fetchall(), columns = ['geometry'])
gdfAggHull['geometry'] = shapely.from_wkt(gdfAggHull['geometry'])
gdfAggHull = gdfAggHull.set_geometry('geometry')
gdfAggHull.set_crs("EPSG:3857", inplace=True)
gdfAggHull

,geometry
0,"POLYGON ((-10872367.066 3529555.461, -10875678..."


#### Visualize the points and the two spatial aggregates you've created
Visualize the points and the two spatial aggregates you've created. The original locations are shown in blue, and the aggregate centroid and aggregate convex hull are shown in red.

In [26]:
gdfPoints['geometry'] = shapely.from_wkt(gdfPoints['geometry'])
gdfPoints = gdfPoints.set_geometry('geometry')
gdfPoints.set_crs("EPSG:3857", inplace=True)

TypeError: Expected bytes or string, got Point

In [25]:
# visualize the original locations and aggregates
m = gdfPoints.explore(tiles="CartoDB positron",
                       style_kwds={"color":"blue","fillColor":"blue"})
m = gdfAggHull.explore(m=m,
                       style_kwds={"color":"red","fillOpacity":"0"} )
m = gdfAggCent.explore(m=m,
                       marker_kwds={"radius":"8"},
                      style_kwds={"color":"red","fillColor":"red","fillOpacity":".7"} )
m

AttributeError: You are calling a geospatial method on the GeoDataFrame, but the active geometry column to use has not been set. 
There are columns with geometry data type (['geometry']), and you can either set one as the active geometry with df.set_geometry("name") or access the column as a GeoSeries (df["name"]) and call the method directly on it.

### Prep for Cluster Detection
To start import libraries needed for detecting spatiotemporal clusters. The main library is st_dbscan. Also, the pandas and numpy libraries are required for configuration of the input to st_dbscan.

In [ ]:
import pandas as pd
import numpy as np
from st_dbscan import ST_DBSCAN

#### Prep for Spatiotemportal Cluster detection
Then, let's run through a simple example of detecting spatiotemporal clusters. Run the following to create a GeoDataFrame with some locations each having epoch time and an ID. The ST_DBSCAN library requires that coordinates be in the same unit as distance measurement. Therefore, we will then convert the coordinate system from longitude/latitude to projected x/y coordinates based on meters. The input to ST_DBSCAN is a Numpy array, so  convert the GeoDataFrame to a Numpy array.

In [ ]:
# create a GeoDataFrame including geometry and epoch date
gdf = gpd.GeoDataFrame({
    "id": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
    "epoch_date": [1704096000, 1687881600, 1687968000, 1688054400, 1688140800, \
                   1688227200, 1672656000, 1672742400, 1672828800,  1016730016, \
                   1673001600, 1673001600, 1672915200, 673001600, 1688054400],
    "geometry": ["POINT(-115.2368 36.2650)",
                "POINT(-115.1356 36.1823)",
                "POINT(-115.1492 36.1779)",
                "POINT(-115.1385 36.1910)",
                "POINT(-115.1256 36.1804)",
                "POINT(-115.1329 36.1735)",
                "POINT(-115.1711 36.1212)",
                "POINT(-115.1656 36.1228)",
                "POINT(-115.1782 36.1221)",
                "POINT(-115.1695 36.1253)",
                "POINT(-115.1790 36.1254)",
                "POINT(-115.1388 36.1858)",
                "POINT(-115.1669 36.1176)",
                "POINT(-115.1755 36.1199)",
                "POINT(-115.1297 36.1900)",
    ],})
# convert to Shapely geometries
gdf['geometry'] = shapely.from_wkt(gdf['geometry'])
# assign longitude/latitude coordinate system
gdf = gdf.set_crs(4326)
gdf

In [ ]:
# convert to projected x/y coordinates as required for st_dbscan
gdf = gdf.to_crs(3857)
gdf

In [ ]:
# Convert to pandas dataframe
df = pd.DataFrame(data={'time': gdf.epoch_date, 'x': gdf.geometry.x, 'y': gdf.geometry.y, 'id':  gdf.id})
data = df.values
# Convert to numpy array
data = np.int_(data)
data

#### Detect spatiotemporal clusters
ST_DBSCAN is a variation of the Density-Based Spatial Clustering of Applications with Noise (DBSCAN) algorithm that is extended to work with spatial data. The parameters are the thresholds for clusters; eps1 is the distance threshold in the units of the coordinate system (meters), eps2 is the time threshold in seconds, and min-samples is the threshold for minimum of items. Run the following to detect clusters where the thresholds are 5 or more items within 5KM and roughly 1 month. The result is an integer label for each input item. Each label >=0 represents a cluster. The label -1 indicates the item is not part of a cluster. Run the following cells to detect spatiotemporal clusrters, review the distinct set of resulting labels, add the integer label to the GeoDataFrame, and finally visualize the clusters.


In [ ]:
# detect clusters 
st_cluster = ST_DBSCAN(eps1 = 5000, eps2 = 3000000, min_samples = 5)
st_cluster.fit(data)

In [ ]:
# list the distinct cluster labels
np.unique(st_cluster.labels)

In [ ]:
# add cluster label to GeoDataFrame
df = pd.DataFrame(data={'id': df.id, 'label': st_cluster.labels})
label_mapping_dict = dict(zip(df["id"], df["label"]))
gdf["label"] = gdf["id"].map(label_mapping_dict)
gdf

In [ ]:
# visualize clusters
gdf.explore(column="label", categorical="True", tiles="CartoDB positron", \
            cmap=['sienna','blue','limegreen'], marker_kwds={"radius":4}, \
            style_kwds={"fillOpacity":1})

#### Create table for labels
The result of cluster detection is a "label" for every data item indicating if the item is part of a cluster, and if so which cluster. You will perform cluster analysis and save the results to the database for further analysis. Run the following to create a database table that will store cluster labels.

In [ ]:
# create table to store cluster detection results (labelled transactions)
cursor.execute("CREATE TABLE transaction_labels (trans_id integer, label integer)")

#### Prep for spatiotemporal cluster detection for customers
Run the following to set a variable for the customer id for analysis, create a GeoDataframe of customer's transactions, and convert to Numpy array for cluster detection. Notice the binding syntax in the WHERE clause (cust_id=:cust) supported by the python-oracledb driver. Then, convert your GeoDataFrame to a numpy array.

In [ ]:
# set customer id for analysis
cust=1

In [ ]:
# create a GeoDataFrame of customer's transactions
cursor.execute("""
 SELECT a.cust_id,  a.trans_id, a.trans_epoch_date,
       (lonlat_to_proj_geom(b.lon,b.lat)).get_wkt()
 FROM transactions a, locations b
 WHERE a.location_id=b.location_id
 AND cust_id=:cust""", cust=cust)
gdf = gpd.GeoDataFrame(cursor.fetchall(), columns = ['cust_id', 'trans_id', 'epoch_date', 'geometry'])
gdf['geometry'] = shapely.from_wkt(gdf['geometry'])
gdf.head()

In [ ]:
# first convert to pandas dataframe
df = pd.DataFrame(data={'time': gdf.epoch_date, 'x': gdf.geometry.x, 'y': gdf.geometry.y, 'trans_id':  gdf.trans_id, 'cust_id':gdf.cust_id})
df.head()

In [ ]:
# then convert to numpy array for st_dbscan
data = df.values
data = np.int_(data)
data[1:10]

#### Detect spatiotemporal clusters

You are now ready to detect spatiotemporal clusters for customer cust_id = 1. The operation accepts three threshold parameters: distance, time, and minimum number of items. Items with neighbors within the distance and time thresholds are considered part of a cluster, and there most be at least the minimum number of items to qualify as a cluster. Distance is in the units of the coordinate system, which in this case is meters. Time is in seconds. Run the following to detect clusters where the thresholds are 5 or more items within 5KM and roughly 1 month. The result is an integer label for each input item. Each label >=0 represents a cluster. The label -1 indicates the item is not part of a cluster. Review the distinct set of resulting labels. Then, add the cluster labels to transactions and print the first several rows. Each transaction is labelled with either -1 (meaning not part of a cluster) or an integer >=0 (meaning the cluster the item belongs to).

In [ ]:
# perform cluster detection
st_cluster = ST_DBSCAN(eps1 = 5000, eps2 = 3000000, min_samples = 5)
st_cluster.fit(data)

In [ ]:
# list the distinct cluster labels
np.unique(st_cluster.labels)

In [ ]:
# add the cluster labels to the DataFrame as a column 
df = pd.DataFrame(data={'trans_id': df.trans_id, 'label': st_cluster.labels})
df.head()

#### Visualize spatiotemporal clusters for customers
Detecting anomalies will require database queries involving the cluster labels. So run the following to insert the the current customer's labelled transactions to the TRANSACTION_LABELS table created in previous steps. Then, retrieve the current customer's transactions with their cluster labels. Then visualize the current customer's labelled transactions. In this case you include the parameter for color coding the items based on cluster label. You may also mouse over an item to see its attributes including the cluster label.

In [ ]:
# insert the labeled transactions to transaction_labels table
cursor.executemany("""
 INSERT INTO transaction_labels
 VALUES (:1, :2)""",
 list(df[['trans_id','label']].itertuples(index=False, name=None)))
connection.commit()

In [ ]:
# labelled transactions for customer
cursor.execute("""
 SELECT a.cust_id, a.location_id, a.trans_id, a.trans_epoch_date,
        (lonlat_to_proj_geom(b.lon,b.lat)).get_wkt(), c.label
 FROM transactions a, locations b, transaction_labels c
 WHERE a.location_id=b.location_id
 AND a.trans_id=c.trans_id
 """)
gdf = gpd.GeoDataFrame(cursor.fetchall(), columns = ['cust_id', 'location_id', 'trans_id', 'trans_epoch_date', 'geometry','label'])
gdf['geometry'] = shapely.from_wkt(gdf['geometry'])
gdf = gdf.set_crs(3857)
gdf.head()

In [ ]:
# visualize labelled transactions for customer
gdf.explore(column="label", categorical="True", tiles="CartoDB positron", \
            marker_kwds={"radius":4}, style_kwds={"fillOpacity":1})

### Detect Anomolies

Run the following to create aggregate centroids for the current customer's spatiotemporal clusters with attributes for cluster label, time range, and number of transactions in the cluster. Observe the first customer has only 1 cluster (label = 0). Then, visualize the spatiotemporal cluster centroid and mouse over the centroid to see its attributes.

In [ ]:
# create GeoDataFrame with cluster centroids for customer
cursor = connection.cursor()
cursor.execute("""
 SELECT label, min(trans_epoch_date) as min_time, max(trans_epoch_date) as max_time,
         SDO_AGGR_CENTROID(
          SDOAGGRTYPE(lonlat_to_proj_geom(b.lon,b.lat), 0.005)).get_wkt() as geometry,
         count(*) as trans_count
 FROM transactions a, locations b, transaction_labels c
 WHERE a.location_id=b.location_id
 AND a.trans_id=c.trans_id
 AND c.label != -1
 GROUP BY label
       """)
gdf = gpd.GeoDataFrame(cursor.fetchall(), columns = ['label','min_time','max_time','geometry','trans_count'])
gdf['geometry'] = shapely.from_wkt(gdf['geometry'])
gdf = gdf.set_crs(3857)
gdf.head()

In [ ]:
# visualize cluster centroids for customer
gdf.explore(tiles="CartoDB positron", marker_kwds={"radius":4})

#### Identify current customer transactions within the time/location range
To identify current customer transactions within the time range of cluster(s) and located at a distance greater than a threshold, you will run a query using WITH … AS … SELECT .. WHERE… syntax as follows. 
```
WITH
    x as ( [transactions] ),
    y as ( [spatiotemporal cluster aggregate centroids] )
SELECT [transaction, cluster label, distance from cluster aggregate centroid, ...]
FROM x, y
WHERE [transaction time within cluster time frame]
AND [distance from cluster > threshold]

```
  
Run the following to return suspicious transactions along with the associated cluster label and distance from the cluster. Then, visualize the spatiotemporal cluster(s) as blue markers and associated suspicious outlier(s) as red markers. Hover over the suspicious transaction(s) to see their attributes.

In [ ]:
# identify anomalies (suspicious transactions) for customer
cursor = connection.cursor()
cursor.execute("""
WITH
   x as (
       SELECT a.cust_id, a.location_id, a.trans_id, a.trans_epoch_date,
              lonlat_to_proj_geom(b.lon,b.lat) as proj_geom, c.label
       FROM transactions a, locations b, transaction_labels c
       WHERE a.location_id=b.location_id
       AND a.trans_id=c.trans_id ),
   y as (
       SELECT label, min(trans_epoch_date) as min_time, max(trans_epoch_date) as max_time,
              SDO_AGGR_CENTROID(
                  SDOAGGRTYPE(lonlat_to_proj_geom(b.lon,b.lat), 0.005)) as proj_geom,
              count(*) as trans_count
       FROM transactions a, locations b, transaction_labels c
       WHERE a.location_id=b.location_id
       AND a.trans_id=c.trans_id
       AND c.label != -1
       GROUP BY label)
 SELECT x.cust_id, x.trans_epoch_date, (x.proj_geom).get_wkt(), x.trans_id, x.label, y.label,
        round(sdo_geom.sdo_distance(x.proj_geom, y.proj_geom, 0.05, 'unit=KM'))
 FROM x, y
 WHERE x.trans_epoch_date between y.min_time and y.max_time
 AND x.label!=y.label
 AND x.label=-1
 AND sdo_within_distance(x.proj_geom, y.proj_geom, 'distance=500 unit=KM') = 'FALSE'
       """)
gdfAnomaly = gpd.GeoDataFrame(cursor.fetchall(), columns = ['cust_id','trans_epoch_date','geometry', 'trans_id','label','outlier_to_label','distance'])
gdfAnomaly['geometry'] = shapely.from_wkt(gdfAnomaly['geometry'])
gdfAnomaly = gdfAnomaly.set_crs(3857)
gdfAnomaly.head()

In [ ]:
# visualize suspicious transactions for customer
m = gdf.explore(tiles="CartoDB positron", marker_type='circle_marker',marker_kwds={"radius":"5"},
                style_kwds={"color":"blue","fillColor":"blue", "fillOpacity":"1"})
m = gdfAnomaly.explore(m=m, marker_type='circle_marker', marker_kwds={"radius":"5"},
                       style_kwds={"color":"red","fillColor":"red", "fillOpacity":"1"} )
m.fit_bounds(m.get_bounds())
m

#### Repeat the process for other customers
To repeat the process for other customer's transactions you could scroll up to the cell where customer ID is set, update to a different customer ID, and rerun the subsequent cells. However it is more convenient to use a script that runs all of the steps.  

Follow Lab 7, Task 4, Step 5-6 of your Workshop to load a script to run all the steps for anomaly detection.
  
The functions in the script will reproduce the previous steps starting from Task 3 after emptying the TRANSACTION_LABELS table as a new set of labels.
- create_connection() establishes a database connection
- get_cluster_centroids( ) detects spatiotemporal transaction clusters for a customer
- get_anomalies( ) identifies suspicious transactions based on overlapping time and distance beyond threshold from clusters
- get_map( ) returns a map of clusters and associated suspicious transactions
  
Run through the following steps to import the script, analyze othercustomer's transactions using functions in the script, and detect suspicious transactions. Repeat these steps with other customer IDs (1-10)


In [ ]:
from anomaly_detection import *

In [ ]:
# set the customer id for analysis
cust = 2

In [ ]:
# run functions to detect anomalies (suspicious transactions)
create_connection()
gdf = get_cluster_centroids(cust)
gdfAnomaly = get_anomalies(cust)
m = get_map()

In [ ]:
# list the spatiotemporal transaction clusters for customer
gdf

In [ ]:
# list the anomalies (suspicious transactions) for customer
gdfAnomaly

In [ ]:
# visualize the anomalies (suspicious transactions) for customer
m.fit_bounds(m.get_bounds())
m

To detect suspicious for other customers, rerun the last 5 cells starting with a different value for the cust variable.

#### Cleanup (Optional)
You may run the following to reset your Autonomous Database to pre-workshop state.

In [ ]:
# drop all database artifacts created in this hands-on lab
import oracledb
my_pwd = open('./my-pwd.txt','r').readline().strip()
my_dsn = open('./my-dsn.txt','r').readline().strip()
connection = oracledb.connect(user="admin", password=my_pwd, dsn=my_dsn)
cursor = connection.cursor()
cursor.execute("drop table transactions")
cursor.execute("drop table locations")
cursor.execute("drop table transaction_labels")
cursor.execute("drop function lonlat_to_proj_geom")
cursor.execute("delete from user_sdo_geom_metadata")
connection.commit()

We hope this workshop has been informative and that you further explore the spatial features of Oracle Database and their use in machine learning and AI workflows.